**Part 1: Preprocess data**

Load packages

In [ ]:
#load in packages
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Check to see which devices are availible for running network on
tf.config.get_visible_devices()

In [ ]:
#check if GPU available to use
#The GPU in the Kaggle online 
if 'GPU' in str(device_lib.list_local_devices()):
    config = tf.compat.v1.ConfigProto(device_count = {'GPU': 0})
    sess = tf.compat.v1.Session(config=config) 

Kaggle has online GPUs that can be used, but they are time limited. Before running the notebook, turn the GPU on in the GUI on the right side of the notebook. 

**Part 2: Get the data**

In [ ]:
#read training data and drop the meta data
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train = train.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test = test.drop(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis=1)

In [ ]:
#quick view of training data
train

In [ ]:
#functions to add image paths to data frames for easy loading of images
def train_id_to_path(x):
    return '../input/petfinder-pawpularity-score/train/' + x + ".jpg"
def test_id_to_path(x):
    return '../input/petfinder-pawpularity-score/test/' + x + ".jpg"

In [ ]:
#also need to add .jpg extensions to image paths
train["img_path"] = train["Id"].apply(train_id_to_path)
test["img_path"] = test["Id"].apply(test_id_to_path)

In [ ]:
#quick view of training data with image paths included
train

**Part 3: Pre-processing image data**

In [ ]:
#Set the size image you want to use
image_height = 128
image_width = 128

#Function that generates tensor
def generate_tensor(image_path):
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_height, image_width))
    return image

In [ ]:
#get all the images in the training folder and put their tensors in a list
X = []
for img in train['img_path']:
    new_img_tensor = generate_tensor(img)
    X.append(new_img_tensor)

In [ ]:
#same for the test images
X_submission = []
for img in test['img_path']:
    new_img_tensor = generate_tensor(img)
    X_submission.append(new_img_tensor)

In [ ]:
#convert X to array
X = np.array(X)

In [ ]:
#convert X_submission to array
X_submission = np.array(X_submission)

Now that all of our data is pre-processed and in tensor form, we can build the model

**Part 3: split data into train and test**

In [ ]:
#generate training data
y = train['Pawpularity']

In [ ]:
#split into training and testing data using a 90-10 split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7)

In [ ]:
x_train.shape

**Part 4: Define the model layers.**

This is a SqueezeNet, which achieved the same accuracy as AlexNet, but is much smaller (reference: https://arxiv.org/pdf/1602.07360.pdf)

A fire module is a building block for CNN archetecture of the SqueezeNet which feeds 1x1 convolutional layers into an expanded layer with 1x1 and 3x3 convolutional layers

In [ ]:
#define the fire_module
def fire_module(x,s1,e1,e3):
    s1x = tf.keras.layers.Conv2D(s1,kernel_size = 1, padding = 'same')(x)
    s1x = tf.keras.layers.ReLU()(s1x)
    e1x = tf.keras.layers.Conv2D(e1,kernel_size = 1, padding = 'same')(s1x)
    e3x = tf.keras.layers.Conv2D(e3,kernel_size = 3, padding = 'same')(s1x)
    x = tf.keras.layers.concatenate([e1x,e3x])
    x = tf.keras.layers.ReLU()(x)
    return x

In [ ]:
#model layers
inputs = tf.keras.Input(shape=(image_height,image_width,3))
x = inputs
nclasses=1
x = tf.keras.layers.Conv2D(96,kernel_size=(7,7),strides=(2,2),padding='same')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
x = fire_module(x, s1 = 16, e1 = 64, e3 = 64) #2
x = fire_module(x, s1 = 16, e1 = 64, e3 = 64) #3
x = fire_module(x, s1 = 32, e1 = 128, e3 = 128) #4
x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
x = fire_module(x, s1 = 32, e1 = 128, e3 = 128) #5
x = fire_module(x, s1 = 48, e1 = 192, e3 = 192) #6
x = fire_module(x, s1 = 48, e1 = 192, e3 = 192) #7
x = fire_module(x, s1 = 64, e1 = 256, e3 = 256) #8
x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides = (2,2))(x)
x = fire_module(x, s1 = 64, e1 = 256, e3 = 256) #9
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Conv2D(nclasses,kernel_size = 1)(x)
output = tf.keras.layers.AveragePooling2D(pool_size=(7,7))(x)
model = tf.keras.Model(inputs, output)


In [ ]:
#compile the model 
model.compile(loss = 'mse', optimizer = 'Adam', metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

**Part 5: Fit the model with the training data**

This trick came from a tutorial which will help with randomness of orientation, zoom, and size shifts of images because not all images in the training set are oriented the same way (source: https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1)

In [ ]:
#augment data
data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

Fit the model using training data

In [ ]:
#fit the model with training data
history = model.fit(
    data_augmentation.flow(x_train,y_train,batch_size=32),
    validation_data = (x_test,y_test),
    steps_per_epoch = len(x_train) // 32,
    epochs = 50
)

**Part 6: Generate Predictions on Test Set**

In [ ]:
#predict on submission
y_pred = model.predict(X_submission)

In [ ]:
#reshape y_pred
y_pred = y_pred.reshape(8,1)

In [ ]:
#convert to dataframe 
y_pred_df = pd.DataFrame()
y_pred_df['Id'] = test['Id']
y_pred_df['Pawpularity'] = y_pred

In [ ]:
#submit to csv
y_pred_df.to_csv('submission.csv',index=False)

In [ ]:
#plot the training and validation RMSE for each epoch
plt.figure()
plt.plot(history.history["rmse"], label="Train RMSE")
plt.plot(history.history["val_rmse"], label="Validation RMSE")
plt.xlabel("Epoch #")
plt.ylabel("RMSE")
plt.title("RMSE for each Epoch")
plt.legend(loc="upper right")

In [ ]:
#plot the training and validation RMSE for each epoch
plt.figure()
plt.plot(history.history["loss"])
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.title("Loss for each Epoch")